# Combining Overland Flow and Simple Crop

## Running Overland Flow on its Own

The overland flow model routes water over a landscape and determines how much water infiltrates particular cells in response to rain events and elevation data. Rainfall in this model is assumed to occur instantaneously and rainfall events are assumed to be independent (so a large rainfall event on a previous day has no bearing on the present day).

In order to run the model we first need to import some source and sink types as well as classes for calling off to CLI models and building requests.

In [2]:
import itertools
import os.path
import netCDF4
from meillionen.client import ClientFunctionModel
from meillionen.resource import PandasHandler, FuncRequest
from meillionen.resource import \
    FileResource, FeatherResource, FeatherResource, NetCDFResource, ParquetResource
from prefect import task, Flow


os.environ['SIMPLECROP'] = 'simplecrop'
os.environ['RUST_BACKTRACE'] = '1'
BASE_DIR = '../../examples/crop-pipeline'

INPUT_DIR = os.path.join(BASE_DIR, 'workflows/inputs')
OUTPUT_DIR = os.path.join(BASE_DIR, 'workflows/outputs')

 We also need to create sources and sinks to describe our data

In [3]:
elevation = FileResource(os.path.join(INPUT_DIR, "hugo_site.asc"))

weather = FeatherResource(os.path.join(INPUT_DIR, "rainfall.feather"))

swid = NetCDFResource(
    path=os.path.join(OUTPUT_DIR, "swid.nc"),
    variable="soil_water_infiltration__depth",
    dimensions=["x", "y", "time"]
)

and build a request to call our model with.

In [7]:
sources = {
    'elevation': elevation,
    'weather': weather
}

sinks = {
    'soil_water_infiltration__depth': swid
}

overlandflow = ClientFunctionModel.from_path(path=os.path.join(BASE_DIR, 'overlandflow/model.py'))

Then call the overland flow model with our request (which will create files on the file system)

In [9]:
overlandflow(sources=sources, sinks=sinks)

## Running Simple Crop on its Own

`simplecrop`  is a model of yearly crop growth that operates on the command line. When called in the current directory with no arguments it expects that there is an data folder to be present. It expects that the data folder contains five files

- `irrig.inp` - daily irrigation
- `plant.inp` - plant growth parameters for the simulation
- `simctrl.inp` - simulation reporting parameters
- `soil.inp` - soil characteristic parameters for the simulation
- `weather.inp` - daily weather data (with variables like maximum temperature, solar energy flux)

`simplecrop` also expects there to be an output folder. After the model has run it will populate the output folder with three files

- `plant.out`- daily plant characteristics
- `soil.out` - daily soil characteristics
- `wbal.out` - summary soil and plant statistics about simulation

In order to wrap this model in an interface that allows you to run the model without manually building those input files and manually converting the output files into a format conducive to analysis we need to have a construct the input files and parse the output files. Fortunately we have such a model wrapper already.

In [12]:
from meillionen.client import ClientFunctionModel
import pandas as pd

run_simple_crop = ClientFunctionModel.from_path('simplecrop_omf')

In [13]:
daily = FeatherResource(os.path.join(BASE_DIR, 'simplecrop/data/daily.feather'))
yearly = FeatherResource(os.path.join(BASE_DIR, 'simplecrop/data/yearly.feather'))
plant = FeatherResource(os.path.join(OUTPUT_DIR, 'plant.feather'))
soil = FeatherResource(os.path.join(OUTPUT_DIR, 'soil.feather'))
tempdir = FileResource('tmp')

sources = {
    'daily': daily,
    'yearly': yearly
}

sinks = {
    'plant': plant,
    'soil': soil,
    'tempdir': tempdir
}

In [14]:
run_simple_crop(sources=sources, sinks=sinks)

In [15]:
soil_df = pd.read_feather(os.path.join(OUTPUT_DIR, 'soil.feather'))
soil_df

,soil_daily_drainage,soil_daily_infiltration,soil_daily_runoff,soil_evaporation,soil_evapotranspiration,soil_water_deficit_stress,soil_water_excess_stress,soil_water_profile_ratio,soil_water_storage_depth,plant_potential_transpiration,day
0,0.00,0.0,0.0,0.00,0.00,1.000,1.0,1.700,246.500000,0.00,0
1,1.86,0.0,0.0,2.23,2.25,1.000,1.0,1.800,260.970001,0.02,3
2,2.25,0.0,0.0,2.62,2.64,1.000,1.0,1.821,264.089996,0.02,6
3,0.94,0.0,0.0,1.31,1.32,1.000,1.0,1.749,253.639999,0.01,9
4,0.57,0.0,0.0,2.53,2.56,1.000,1.0,1.718,249.089996,0.02,12
...,...,...,...,...,...,...,...,...,...,...,...
95,0.00,0.0,0.0,0.61,4.04,0.566,1.0,1.067,154.759995,1.55,282
96,0.00,0.0,0.0,0.26,1.64,0.544,1.0,1.049,152.050003,0.56,285
97,0.00,0.0,0.0,0.46,2.92,0.499,1.0,1.011,146.649994,0.88,288
98,0.00,0.0,0.0,0.75,4.84,0.449,1.0,0.971,140.729996,1.25,291


## Using Prefect to Feed Overland Flow Results into Simple Crop

In [17]:
overlandflow = ClientFunctionModel.from_path(os.path.join(BASE_DIR, 'overlandflow/model.py'))
simplecrop = ClientFunctionModel.from_path('simplecrop_omf')


@task()
def run_overland_flow():
    elevation = FileResource(os.path.join(INPUT_DIR, "hugo_site.asc"))
    weather = FeatherResource(os.path.join(INPUT_DIR, "rainfall.feather"))
    swid = NetCDFResource(
        path=f"{OUTPUT_DIR}/swid.nc",
        variable="soil_water_infiltration__depth",
        dimensions=['x', 'y', 'time']
    )

    sources = {
        'elevation': elevation,
        'weather': weather
    }
    sinks = {
        'soil_water_infiltration__depth': swid
    }

    overlandflow(sources=sources, sinks=sinks)

    return swid


@task()
def chunkify_soil_water_infiltration_depth(swid):
    swid = swid.to_dict()
    ds = netCDF4.Dataset(swid['path'], 'r')
    variable = ds[swid['variable']]
    x_d, y_d, time_d = variable.get_dims()
    return [{'soil_water_infiltration__depth': variable[x, y, :], 'x': x, 'y': y} for (x,y) in itertools.product(range(10,12), range(21,23))]


@task()
def convert_swid_chunk_to_simplecrop(c):
    return {'daily': daily, 'yearly': yearly, **c}


@task()
def simplecrop_process_chunk(data):
    yearly = data['yearly']
    daily = data['daily']
    soil_water_infiltration__depth = data['soil_water_infiltration__depth']
    x = data['x']
    y = data['y']
    df = PandasHandler(simplecrop.source('daily')).load(daily)
    df['rainfall'] = soil_water_infiltration__depth
    daily_resource = FeatherResource(os.path.join(OUTPUT_DIR, f'outputs/daily/{x}/{y}.feather'))
    PandasHandler(simplecrop.source('daily')).save(daily_resource, data=df)

    plant = ParquetResource(os.path.join(OUTPUT_DIR, f'plant/{x}/{y}/data.parquet'))
    soil = ParquetResource(os.path.join(OUTPUT_DIR, f'soil/{x}/{y}/data.parquet'))
    tempdir = FileResource(os.path.join(OUTPUT_DIR, f'temp/{x}/{y}'))
    
    sources = {
        'daily': daily_resource,
        'yearly': yearly
    }
    
    sinks = {
        'plant': plant,
        'soil': soil,
        'tempdir': tempdir
    }

    simplecrop(sources=sources, sinks=sinks)


daily = FeatherResource(os.path.join(BASE_DIR, 'simplecrop/data/daily.feather'))
yearly = FeatherResource(os.path.join(BASE_DIR, 'simplecrop/data/yearly.feather'))


with Flow('crop_pipeline') as flow:
    overland_flow = run_overland_flow()
    surface_water_depth_chunks = chunkify_soil_water_infiltration_depth(overland_flow)
    simplecrop_chunks = convert_swid_chunk_to_simplecrop.map(surface_water_depth_chunks)
    yield_chunks = simplecrop_process_chunk.map(simplecrop_chunks)

flow.run()


[2021-05-04 20:54:21-0700] INFO - prefect.FlowRunner | Beginning Flow run for 'crop_pipeline'
[2021-05-04 20:54:21-0700] INFO - prefect.TaskRunner | Task 'run_overland_flow': Starting task run...
[2021-05-04 20:54:26-0700] INFO - prefect.TaskRunner | Task 'run_overland_flow': Finished task run for task with final state: 'Success'
[2021-05-04 20:54:26-0700] INFO - prefect.TaskRunner | Task 'chunkify_soil_water_infiltration_depth': Starting task run...
[2021-05-04 20:54:26-0700] INFO - prefect.TaskRunner | Task 'chunkify_soil_water_infiltration_depth': Finished task run for task with final state: 'Success'
[2021-05-04 20:54:26-0700] INFO - prefect.TaskRunner | Task 'convert_swid_chunk_to_simplecrop': Starting task run...
[2021-05-04 20:54:26-0700] INFO - prefect.TaskRunner | Task 'convert_swid_chunk_to_simplecrop': Finished task run for task with final state: 'Mapped'
[2021-05-04 20:54:26-0700] INFO - prefect.TaskRunner | Task 'convert_swid_chunk_to_simplecrop[0]': Starting task run...
[

<Success: "All reference tasks succeeded.">